In [ ]:
#In this notebook we will take generic ephemeris data and use it to display a trail in OpenSpace
#In order to do this we will first take the generic data and convert it into a NAIF SPICE file (.bsp)
# which is the highest precision type of data file OpenSpace can import. Once we have created the 
# spice data file we generate an OpenSpace asset file with paramaters describing how to visualize the data.
# Finally we send a command to OpenSpace to add the asset file to the running instance of OpenSpace for visualiztion.

In [12]:
#generate data for trail and output as states for input to mkspk

data_name = '367943 Duende (2012 DA14)'
data_name_strip = '367943_2012_DA14'
data_id = '367943'
#import pandas as pd

#propagated = pd.read_csv("example_states.csv", index_col=False)

#print(propagated)

In [13]:
#create mkspk setupfile and comment
setup = ""

setup += "\\begindata\n\
   INPUT_DATA_TYPE   = 'STATES'\n\
   INPUT_DATA_FILE   = 'da14example_states.csv'\n\
   OUTPUT_SPK_FILE   = '" + data_name_strip + ".bsp'\n\
   OUTPUT_SPK_TYPE   = 8\n\
   OBJECT_ID         = " + data_id + "\n\
   OBJECT_NAME       = '" + data_name_strip + "'\n\
   CENTER_ID         = 0\n\
   CENTER_NAME       = 'SSB'\n\
   REF_FRAME_NAME    = 'GALACTIC'\n\
   PRODUCER_ID       = 'B612'\n\
   DATA_ORDER        = 'EPOCH X Y Z VX VY VZ'\n\
   INPUT_DATA_UNITS  = ('ANGLES=DEGREES' 'DISTANCES=AU')\n\
   DATA_DELIMITER    = ','\n\
   LINES_PER_RECORD  = 1\n\
   IGNORE_FIRST_LINE = 1\n\
   LEAPSECONDS_FILE  = 'naif0012.tls'\n\
   POLYNOM_DEGREE    = 9\n\
   SEGMENT_ID        = 'SPK_STATES_08'\n\
   TIME_WRAPPER      = '# ETSECONDS'\n\
\\begintext\n"

setup_file = open("setup.mkspk", "w")
setup_file.write(setup)
setup_file.close()


In [14]:
#run mkspk
import os
os.system('rm ' + data_id + '.bsp')
os.system('mkspk.exe -setup setup.mkspk')


0

In [15]:
#create .asset file referencing mkspk output 
asset = "local assetHelper = asset.require('util/asset_helper')\n"
asset += "local spkTrail = {\n\
    Identifier = '" + data_name + "',\n\
    Parent = 'Earth',\n\
    Renderable = {\n\
        Type = 'RenderableTrailTrajectory',\n\
        Enabled = true,\n\
        Translation = {\n\
          Type = 'SpiceTranslation',\n\
          Target = '" + data_id + "',\n\
          Observer = 'SSB',\n\
          Kernels = {asset.localResource('"+data_name_strip+".bsp')}\n\
        },\n\
        Color = { 0.4, 0.0, 0.7 },\n\
        StartTime = '2020-05-31 00:01:09.185',\n\
        EndTime = '2022-08-09 00:01:09.183',\n\
        SampleInterval = 3600\n\
    },\n\
    GUI = {\n\
        Name = '" + data_name + "',\n\
        Path = '/B612'\n\
    }\n\
}\n\n"

asset += "assetHelper.registerSceneGraphNodesAndExport(asset, { spkTrail })"
    
asset_file = open(data_name_strip+".asset", "w")
asset_file.write(asset)
asset_file.close()

In [16]:
#send command to add asset

#get full path for script command
import os
asset_full_path = os.path.realpath(asset_file.name)

#connect to openspace 
from websocket import create_connection
import json
ws_endpoint = f"ws://localhost:4682/websocket"
ws = create_connection(ws_endpoint)
#command to add asset
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.asset.add",
                              "arguments": [asset_full_path],
                              "return": False}})
ws.send(message)
#refresh menu
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.setPropertyValueSingle",
                              "arguments": ['Modules.CefWebGui.Reload', None],
                              "return": False}})
ws.send(message)
#ws.close() #uncomment if done and you are not going to remove the asset.

168

In [ ]:
# (optional) remove asset file (needed if you want to change things and try again)
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.asset.remove",
                              "arguments": [asset_full_path],
                              "return": False}})
ws.send(message)
ws.close()